# Requiment 1

## Library

In [2]:
import pandas as pd
import numpy as np
import hashlib
import random
import time

## Data

In [3]:
def read_file(filename):
      with open(filename, 'r') as file:
          text = file.readlines()
      df = pd.DataFrame(text, columns=["Text"])
      return df

## LSH

### Version 1:

In [ ]:
class InMemoryMinHashLSH:

    def __init__(self, documents):
      self.documents = documents
    pass

    def convert_to_shingling(self, text, k):
      shingles = set()
      words = text.split(" ")
      for i in range(len(words) - k + 1):
          tmp = " ".join(words[i:i+k])
          shingles.add(tmp)
      return shingles

    def convert_to_boolean_vector(self, df, text):
      union_shingles = set()  # Initialize as a set
      for _, row in df.iterrows():
          union_shingles |= set(row['Shingles'])  # Use set union operation
      # Convert the set to a numpy array for efficient membership checking
      union = np.array(list(union_shingles))
      result = []
      for shingle in union:
          if shingle in text:
              result.append(1)
          else:
              result.append(0)
      return result


    def shingling(self,df):
      shingles_list = []
      boolean_list = []

      for index, row in df.iterrows():
          text = row['Text'].replace(',',"").replace(';',"").replace('.',"").replace('\'',"")
          shingles = self.convert_to_shingling(text, k=8)  # Adjust k value as needed
          shingles_list.append(shingles)
      shingles_df = pd.DataFrame({'Shingles': shingles_list})

      for index, row in df.iterrows():
          text = row['Text'].replace(',',"").replace(';',"").replace('.',"").replace('\'',"")
          boolean_vector = self.convert_to_boolean_vector(shingles_df,text)  # Adjust k value as needed
          boolean_list.append(boolean_vector)
      boolean_vector_df = pd.DataFrame({'vector': boolean_list})
      return boolean_vector_df

    def convert_to_signature(self,text,arr):
      vector = np.array(text)
      result=[]
      for i in range(1,50):
        random.shuffle(arr)
        for j in range(1,len(arr)):
          if vector[arr.index(j)]==1:
            result.append(arr.index(j))
            break
      return np.array(result)

    def minhashing(self,df):
      signature_list=[]
      for index, row in df.iterrows():
        arr = self.generate_arrHash(len(text))
        text = row['vector']
        minhash = self.convert_to_signature(text,arr)
        signature_list.append(minhash)
      signature_df = pd.DataFrame({'signature': signature_list})

      return signature_df

    def generate_arrHash(self,n):
      arr = [i for i in range(1, n+1)]
      return arr

    def split_signature(self, signature, k):
        result = []
        for i in range(0, len(signature), k):
            tmp = signature[i:i+k].tolist()
            result.append(tmp)
        return result
    # def hash_busket(self,signature_split):


    def truncated_hash(self, vector, length):
        hash_value = hashlib.sha256(bytes(str(vector), 'utf-8')).hexdigest()
        truncated_value = int(hash_value[:length], 16)
        result = {truncated_value: vector}
        return result

    def locality_sensitivity_hashing(self, signature):
        result = {}
        for index, row in signature.iterrows():
            text = row['signature']
            for vector in self.split_signature(text, 2):
                truncated_hash_result = self.truncated_hash(vector, 4)
                for key, value in truncated_hash_result.items():
                    if key not in result:
                        result[key] = []
                    result[key].append(value)
            # break
        return result

    def check_bucket(self,bucket):
      for i in bucket.values():
        if len(i) > 1:
            print(i)

    def run(self):
      shingling_result= self.shingling(self.documents)

      return


    def approxNearestNeighbors(self, key, n):
        pass


### Version 2:

In [ ]:
class InMemoryMinHashLSH:

    def __init__(self, documents):
      self.documents = documents
    pass

    def shingling(self, df):
      def preprocess_text(text):
          text = text.replace(',', "").replace(';', "").replace('.', "").replace('(', "").replace(')', "")
          return text.lower()
      df['Text'] = df['Text'].apply(preprocess_text)

      def convert_to_shingling(text, k):
        shingles = set()
        words = text.split(" ")

        # Compute the hash for the first k-gram
        current_hash = hash(" ".join(words[:k]))
        shingles.add(current_hash)

        # Use rolling hash to efficiently compute hashes for subsequent k-grams
        base = 101  # Choose a suitable base for the rolling hash
        power = base ** (k - 1)

        for i in range(1, len(words) - k + 1):
            # Remove the contribution of the first word from the previous k-gram
            current_hash -= ord(words[i - 1][0]) * power  # Assuming we only take the first character
            # Shift the hash by multiplying base and add the hash of the new word
            current_hash = current_hash * base + ord(words[i + k - 1][0])  # Assuming we only take the first character
            shingles.add(current_hash)

        return shingles

      def convert_to_boolean(text, union):
        result = []
        for shingle in union:
            if shingle in text:
                result.append(1)
            else:
                result.append(0)
        return np.array(result)
      def process1(df, k):
        df['Text'] = df['Text'].apply(lambda text: convert_to_shingling(text, k))

      process1(df,8)

      def process2(df):
          all_shingles = set()
          for text in df['Text']:
            all_shingles.update(text)
          print(len(all_shingles))
          df['vector']=df['Text'].apply(lambda text:convert_to_boolean(text,all_shingles))
          return df

      process2(df)

      return df



    def minhashing(self,df):
      def permute_row(row, permutation):
          return np.array(row)[permutation]

      def permute_dataframe(df, permutation):
          return pd.DataFrame(df['vector'].apply(lambda row: permute_row(row, permutation)))
      k = 50
      signatures = []
      for i in range(k):
          permutation = np.random.permutation(len(df.iloc[0]['vector']))

          result = permute_dataframe(df, permutation)
          signature = []
          for index, row in result.iterrows():
              first_one_index = np.where(row['vector'] == 1)[0][0]
              signature.append(first_one_index)
          signatures.append(signature)

      #Create df['signature']
      #Tranpose signatures then add to signature_df['signature'], signature_df just have 1 column is signature
      signature_array = np.array(signatures).T
      # Create DataFrame with column name 'signature'
      signature_df = pd.DataFrame({'signature': signature_array.tolist()})

      return signature_df

    def locality_sensitivity_hashing(self, signature):
      def split_signature(signature, k):
        result = []
        for i in range(0, len(signature), k):
            tmp = signature[i:i+k]  # Remove .tolist()
            result.append(tmp)
        return result

      def truncated_hash( vector, length):
        hash_value = hashlib.sha256(bytes(str(vector), 'utf-8')).hexdigest()
        truncated_value = int(hash_value[:length], 16)
        result = {truncated_value: vector}
        return result

      result = {}

      def process_row(row):
          text = row['signature']
          hash_results = {}  # Change to a dictionary
          for vector in split_signature(text,2):
              truncated_hash_result = truncated_hash(vector, 8)
              for key, value in truncated_hash_result.items():
                 hash_results.setdefault(key, []).extend((value, row.name))
          return hash_results
      # Apply the process_row function to each row of the DataFrame
      hash_results_list = signature.apply(process_row, axis=1)
      # Aggregate the hash results into the final result dictionary
      for hash_results in hash_results_list:
          for key, value in hash_results.items():
              result.setdefault(key, []).extend(value)
      return result

    def check_bucket(self, bucket, n):
      pair_counts = {}  # Dictionary to store counts of pairs of row_index
      for key, values in bucket.items():
          if len(values) > 2:
              # Collect pairs of row_index values
              row_indexes = values[1::2]
              num_indexes = len(row_indexes)
              for i in range(0, num_indexes, 2):  # Iterate over pairs of row_index
                  if i + 1 < num_indexes:  # Ensure not to go out of range
                      row_index1 = row_indexes[i]
                      row_index2 = row_indexes[i+1]
                      # Skip pairs where both row indexes are the same
                      if row_index1 != row_index2:
                          pair = (row_index1, row_index2)
                          # Update the count for the current pair
                          pair_counts[pair] = pair_counts.get(pair, 0) + 1
                          vector = values[i]

      # Sort pair_counts by count in descending order
      sorted_pair_counts = sorted(pair_counts.items(), key=lambda x: x[1], reverse=True)

      # Print n pairs in descending order by count
      for i, (pair, count) in enumerate(sorted_pair_counts[:n]):
          print(f"  {pair[0]} and {pair[1]} appear together {count} times.")
          if i == n - 1:
              break

    def run(self):

      ###Shingling
      start_time = time.time()
      shingling_result= self.shingling(self.documents)
      end_time = time.time()
      print("part 1(shingling): ", end_time - start_time)
      ##Minhashing

      start_time = time.time()
      sig = self.minhashing(shingling_result)
      end_time = time.time()
      print("part 2(minhashing): ", end_time - start_time)
      ###LSH
      start_time = time.time()
      result = self.locality_sensitivity_hashing(sig)
      end_time = time.time()
      print("part 3(LSH): ", end_time - start_time)

      self.check_bucket(result,3)
      return


    def approxNearestNeighbors(self, key, n):
        pass


### Version 3:

In [12]:
class InMemoryMinHashLSH:

    def __init__(self, documents):
      self.documents = documents
      self.union_shingle = set()
      self.permutation = []
    pass
    def shingling(self, df):
      def preprocess_text(text):
          text = text.replace(',', "").replace(';', "").replace('.', "").replace('(', "").replace(')', "")
          return text.lower()
      df['Text'] = df['Text'].apply(preprocess_text)

      def convert_to_shingling(text, k):
        shingles = set()
        words = text.split(" ")
        for i in range(len(words) - k + 1):
            shingles.add(" ".join(words[i:i+k]))

        return shingles
      def convert_to_boolean(text, union):
        result = []
        for shingle in union:
            if shingle in text:
                result.append(1)
            else:
                result.append(0)
        return np.array(result)
      def process1(df, k):
        df['Text'] = df['Text'].apply(lambda text: convert_to_shingling(text, k))

      process1(df,1)

      def process2(df):
          all_shingles = set()
          if len(self.union_shingle) != 0:
            df['vector']=df['Text'].apply(lambda text:convert_to_boolean(text,self.union_shingle))
          else:
            for text in df['Text']:
              all_shingles.update(text)
            print("Union length: ", len(all_shingles))
            self.union_shingle = all_shingles
            df['vector']=df['Text'].apply(lambda text:convert_to_boolean(text,all_shingles))
          return df

      process2(df)

      return df



    def minhashing(self, df, k=50):
      # Function to permute a row
      def permute_row(row, permutation):
          return np.array(row)[permutation]
      # Generate k permutations and compute signatures
      signatures = []
      num_shingles = len(df.iloc[0]['vector'])
      if len(self.permutation)!=0:
        for i in range(len(self.permutation)):
            permutation = self.permutation[i]
            signature = df['vector'].apply(lambda row: np.where(permute_row(row, permutation) == 1)[0][0]).tolist()
            signatures.append(signature)
        signature_array = np.array(signatures).T
        signature_df = pd.DataFrame({'signature': signature_array.tolist()})
      else:
        for _ in range(k):
            permutation = np.random.permutation(num_shingles)
            self.permutation.append(permutation)
            signature = df['vector'].apply(lambda row: np.where(permute_row(row, permutation) == 1)[0][0]).tolist()
            signatures.append(signature)
        signature_array = np.array(signatures).T
        signature_df = pd.DataFrame({'signature': signature_array.tolist()})

      return signature_df

    def locality_sensitivity_hashing(self, signature):
      def split_signature(signature, k):
        result = []
        for i in range(0, len(signature), k):
            tmp = signature[i:i+k]  # Remove .tolist()
            result.append(tmp)
        return result

      def truncated_hash(vector, length):
        hash_value = hashlib.sha256(bytes(str(vector), 'utf-8')).hexdigest()
        truncated_value = int(hash_value[:length], 16)
        return truncated_value

      result = {}

      def process_row(row):
        text = row['signature']
        hash_results = []
        for vector in split_signature(text, 1):
            truncated_hash_result = truncated_hash(vector, 8)
            hash_results.append(truncated_hash_result)
        return hash_results

      result_df = pd.DataFrame()
      result_df['bucket'] = signature.apply(process_row, axis=1)
      return result_df

    def run(self):

      shingling_result= self.shingling(self.documents)
      sig = self.minhashing(shingling_result)
      result = self.locality_sensitivity_hashing(sig)
      self.bucket = result

      return

    def approxNearestNeighbors(self, key, n):
      shingling_key = self.shingling(key)
      sig_key = self.minhashing(shingling_key)
      result_key = self.locality_sensitivity_hashing(sig_key)
      hash_key = result_key.loc[0,'bucket']
      ## Calculate similarity of hash_key each row in self.bucket
      def calculate_similarity(signature1, signature2):
        set1 = set(signature1)
        set2 = set(signature2)
        intersection = len(set1.intersection(set2))
        union = len(set1.union(set2))
        if union == 0:
            return 0  # Return 0 if union is 0 to avoid division by zero
        else:
            similarity = intersection / union
            return similarity
      result = []
      for index, row in self.bucket.iterrows():
        value = calculate_similarity(hash_key,row['bucket'])
        result.append(value)
      self.result_df = pd.DataFrame({'Similarity': result})
      self.result_df = self.result_df.sort_values(by='Similarity', ascending=False)

      return self.result_df.head(n)


### Running

In [13]:
data = read_file("/content/WebOfScience-5736.txt")
key = read_file("/content/key.txt")
start = time.time()
lsh = InMemoryMinHashLSH(data)
lsh.run()
end = time.time()
print("time run: ", end - start)
result = lsh.approxNearestNeighbors(key,5)
print(result)

Union length:  63061
time run:  211.91343808174133
      Similarity
0       0.426471
4       0.269231
5636    0.195122
826     0.195122
1353    0.192771


In [16]:
data = read_file("/content/WebOfScience-5736.txt")
key = read_file("/content/key.txt")
print(data.loc[4,'Text'])

This paper presents the simulation results of a linear, fully integrated, two-stage digitally programmable 130 nm CMOS power amplifier (PA) operating at 2.4 GHz. Its power stage is composed of a set of amplifying cells which can be enabled or disabled independently by a digital control circuit. All seven operational modes are univocal in terms of 1 dB output compression point (OCP1dB), saturated output power (P-SAT) and power gain at 2.4 GHz. The lowest power mode achieves an 8.1 dBm P-SAT, a 13.5 dB power gain and consumes 171 mW DC power (P-DC) at an OCP1dB of 6 dBm, whereas the highest power mode reaches an 18.9 dBm P-SAT and a 21.1 dB power gain and consumes 415 mW P-DC at an OCP1dB of 18.2 dBm.

